In [387]:
import talib as ta
import pandas as pd
import numpy as np

In [388]:
# read the data from the CSV files
train_data = pd.read_csv('train_data_50.csv') 

# change date into datetime objects
train_data['Date'] = pd.to_datetime(train_data['Date'])

# set indexes 
train_data.set_index(["Ticker", "Date"], inplace=True)

# sort data by tickers
tickers = sorted(train_data.index.get_level_values('Ticker').unique())

In [389]:
# array of all stocks and close prices
open_prices = []

for ticker in tickers:
    stock_close_data = train_data.loc[ticker]["Open"]
    open_prices.append(stock_close_data.values)

open_prices = np.stack(open_prices)
print(open_prices.shape)
print(open_prices)

# array for final return - trades
trades = np.zeros_like(open_prices)
trades

cash = 25000

(50, 1452)
[[17.04294995 17.25096871 16.96448663 ... 67.35797313 68.06231506
  68.62432851]
 [12.20377331 12.24209241 12.04319784 ... 27.93657064 28.10079496
  28.43107009]
 [27.76139563 28.32888582 28.12086706 ... 45.28424827 45.45394932
  45.20031238]
 ...
 [12.73111928 12.80593376 12.77856257 ... 40.5016405  40.69141088
  40.9340999 ]
 [ 8.02149948  8.0908393   8.10908629 ... 45.21855798 46.12362377
  46.14186937]
 [20.12126341 20.38584965 20.38584965 ... 57.08475709 57.0592138
  57.54641278]]


In [390]:
# calculate golden cross value using TA-lib
# golden cross is basically the same as the SMA example given
# tried with EMA, ended up with lower sharpe ratio

for stock in range(len(open_prices)): 
    fast_sma = ta.SMA(open_prices[stock], timeperiod=5)
    slow_sma = ta.SMA(open_prices[stock], timeperiod=20)
    
    talib_rsi = ta.RSI(stock-stock_close_data, timeperiod=25)

    for day in range(1, len(open_prices[0])-1):
        
        # Buy: fast SMA crosses above slow SMA
        if fast_sma[day] > slow_sma[day] and fast_sma[day-1] <= slow_sma[day-1] and talib_rsi[day] < 30:
            # we are trading the next day's open price
            
            if (cash >= open_prices[stock][day] * 30):
                trades[stock][day+1] = 30
                cash -= open_prices[stock][day] * 30

        # Sell/short: fast SMA crosses below slow SMA
        elif fast_sma[day] < slow_sma[day] and fast_sma[day-1] >= slow_sma[day-1] and talib_rsi[day] > 70:
            # we are trading the next day's open price
           
            trades_stock = trades[stock] 
            numStocks = np.sum(trades_stock)
            if (numStocks >= 22):
                trades[stock][day + 1] = -22
                cash += open_prices[stock][day] * 22
    
        # else do nothing
        else:
            trades[stock][day+1] = 0

print(cash)

271.6276181468973


/var/folders/5h/dq1rq3ts62j1zt97ddbg3jbc0000gn/T/ipykernel_8998/1366759498.py:22: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  elif fast_sma[day] < slow_sma[day] and fast_sma[day-1] >= slow_sma[day-1] and talib_rsi[day] > 70:
/var/folders/5h/dq1rq3ts62j1zt97ddbg3jbc0000gn/T/ipykernel_8998/1366759498.py:14: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  if fast_sma[day] > slow_sma[day] and fast_sma[day-1] <= slow_sma[day-1] and talib_rsi[day] < 30:


In [391]:
from eval_algo import eval_actions

portfolio_value, sharpe_ratio = eval_actions(trades, open_prices, cash=25000, verbose=True)
print(f"\nPortfolio value: {portfolio_value}")
print(f"Sharpe ratio: {sharpe_ratio}")

final portfolio value: 46810.22957996058
cash: 192.48504227821445
positions: [0, 30.0, 90.0, 0, 30.0, 60.0, 60.0, 38.0, 30.0, 0, 90.0, 0, 30.0, 30.0, 30.0, 30.0, 30.0, 90.0, 60.0, 0, 30.0, 0, 0, 60.0, 0, 60.0, 30.0, 30.0, 30.0, 30.0, 60.0, 38.0, 30.0, 38.0, 60.0, 30.0, 30.0, 30.0, 30.0, 30.0, 0, 0, 30.0, 60.0, 0, 0, 30.0, 0, 0, 0]
short position info: defaultdict(<class 'list'>, {})
short value: 0

Portfolio value: [25000, 25000, 25000, 25000, 25000, 25000, 25000, 25000, 25000, 25000, 25000, 25000, 25000, 25000, 25000, 25000, 25000, 25000, 25000, 25000, 25000, 25000, 25000, 25000, 25000, 25000, 25000, 25000, 25000, 25000, 25000, 25000, 25000, 24981.09859812466, 24950.376767066, 24891.53600543254, 24915.369169292786, 24902.565829122555, 24927.111498514536, 24936.21450844683, 24937.00513482248, 24903.079847987257, 24908.316680116164, 24913.82051387839, 24895.446913213913, 24909.123156189893, 24927.466268634078, 24907.893353265656, 24916.111724515617, 24931.2977608663, 24932.446101771802,